## Code story for candidate it's about
* get number of mentions of ea candidate / story (raw)
* mark as dominating candidate

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas
%matplotlib inline
import numpy as np

In [2]:
from collections import Counter
import re

In [3]:
df = pandas.read_csv('DATA/GT_10_STORIES/url_body_complete.csv',index_col=0)

In [4]:
# drop nulls
df = df.dropna()
print len(df) # should be 2718 remaining stories

2718


## Find mentions

In [5]:
candids = ['TRUMP', 'CLINTON', 'SANDERS', 'CRUZ']

In [6]:
df['body_upper'] = df.body.apply(lambda row: row.upper())

In [7]:
#df.to_csv('DATA/GT_10_STORIES/url_body_UPPER.csv')

In [8]:
def count_candids(body): 
    count = Counter(re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+", body))
    return {key: count[key] for key in count if key in candids}

In [9]:
def count_positiv(body): 
    count = Counter(re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+", body))
    return len({key: count[key] for key in count if key in positiv}.values())

In [10]:
def top_candid(body): 
    count = Counter(re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+", body))
    c =  Counter({key: count[key] for key in count if key in candids}).most_common(1)
    if c:
        return c[0][0]
    else:
        return None

In [11]:
df['counts'] = df.body_upper.apply(lambda body: count_candids(body))

In [12]:
df['top_candid'] = df.body_upper.apply(lambda body: top_candid(body))

In [13]:
df.top_candid.head()

9      CRUZ
11     CRUZ
20    TRUMP
21     CRUZ
25    TRUMP
Name: top_candid, dtype: object

In [14]:
#df.to_csv("DATA/FOR_CORR/candid_coded.csv")

In [15]:
df.columns

Index([u'url', u'body', u'body_upper', u'counts', u'top_candid'], dtype='object')

In [16]:
emot = pandas.read_csv("DATA/FOR_CORR/all_segments_no_title.csv", index_col=0)
emot.columns

Index([u'_id', u'author', u'sentiment', u'time', u'url', u'a_id',
       u'date_written', u'org', u't_id', u'candidates', u'candid_camp',
       u'num_tweets', u'sentiment_mean', u'num_tweets_all', u'wc',
       u'negativ_ct', u'positiv_ct', u'negativ_percent', u'positiv_percent',
       u'emotionality', u'positivity'],
      dtype='object')

In [17]:
joined = pandas.merge(emot, df, left_on=emot.url, right_on=df.url )

In [18]:
joined.columns

Index([u'_id', u'author', u'sentiment', u'time', u'url_x', u'a_id',
       u'date_written', u'org', u't_id', u'candidates', u'candid_camp',
       u'num_tweets', u'sentiment_mean', u'num_tweets_all', u'wc',
       u'negativ_ct', u'positiv_ct', u'negativ_percent', u'positiv_percent',
       u'emotionality', u'positivity', u'url_y', u'body', u'body_upper',
       u'counts', u'top_candid'],
      dtype='object')

In [19]:
joined.rename(columns={'url_x': 'url'}, inplace=True)

In [20]:
joined[[u'_id', u'author', u'sentiment', u'time', u'url', u'a_id',
       u'date_written', u'org', u't_id', u'candidates', u'candid_camp',
       u'num_tweets', u'sentiment_mean', u'num_tweets_all', u'wc',
       u'negativ_ct', u'positiv_ct', u'negativ_percent', u'positiv_percent',
       u'emotionality', u'positivity',
       u'counts', u'top_candid']].to_csv("DATA/FOR_CORR/candid_coded_all_tweets.csv")

In [21]:
df = joined.drop_duplicates('url')


In [22]:
len(df)

2579

In [23]:
df[[u'_id', u'author', u'sentiment', u'time', u'url', u'a_id',
       u'date_written', u'org', u't_id', u'candidates', u'candid_camp',
       u'num_tweets', u'sentiment_mean', u'num_tweets_all', u'wc',
       u'negativ_ct', u'positiv_ct', u'negativ_percent', u'positiv_percent',
       u'emotionality', u'positivity',
       u'counts', u'top_candid']].to_csv("DATA/FOR_CORR/candid_coded_by_story.csv")